# Video

In [3]:
from ultralytics import YOLO
import cv2
import torch
import numpy as np
from collections import Counter

print("GPU disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Usando:", torch.cuda.get_device_name(0))

model = YOLO("yolov8n.pt")
model_line = YOLO("yolov8n_line_detail.pt")  
model_inclinacion = YOLO("yolo11n_inclinacion.pt")
model.to('cuda' if torch.cuda.is_available() else 'cpu')
model_line.to('cuda' if torch.cuda.is_available() else 'cpu')
model_inclinacion.to('cuda' if torch.cuda.is_available() else 'cpu')
print("Modelo cargado en:", model.device)

GPU disponible: True
Usando: NVIDIA GeForce RTX 4060
Modelo cargado en: cuda:0


In [ ]:
cap = cv2.VideoCapture("SoccerNet/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_720p.mkv")

field_map = {
    # LÍNEAS DE BORDE
    "Side line left":     (0, 0, 0, 68),
    "Side line right":    (105, 0, 105, 68),
    "Side line top":      (0, 68, 105, 68),
    "Side line bottom":   (0, 0, 105, 0),
    "Middle line":        (52.5, 0, 52.5, 68),

    # CÍRCULOS
    "Circle central":     {"center": (52.5, 34), "radius": 9.15},

    # ÁREA GRANDE IZQUIERDA
    "Big rect. left main":   (16.5, 13.85, 16.5, 54.15),
    "Big rect. left top":    (0, 54.15, 16.5, 54.15),
    "Big rect. left bottom": (0, 13.85, 16.5, 13.85),

    # ÁREA GRANDE DERECHA
    "Big rect. right main":   (88.5, 13.85, 88.5, 54.15),
    "Big rect. right top":    (105, 54.15, 88.5, 54.15),
    "Big rect. right bottom": (105, 13.85, 88.5, 13.85),

    # ÁREA CHICA IZQUIERDA
    "Small rect. left main":   (5.5, 24.84, 5.5, 43.16),
    "Small rect. left top":    (0, 43.16, 5.5, 43.16),
    "Small rect. left bottom": (0, 24.84, 5.5, 24.84),

    # ÁREA CHICA DERECHA
    "Small rect. right main":   (99.5, 24.84, 99.5, 43.16),
    "Small rect. right top":    (105, 43.16, 99.5, 43.16),
    "Small rect. right bottom": (105, 24.84, 99.5, 24.84),
}
#punto de el circulo central
circle_points = {
    "Circle central": (52.5, 34),
    "Circle central left": (52.5 - 9.15, 34),
    "Circle central right": (52.5 + 9.15, 34),
    "Circle central top": (52.5, 34 + 9.15),
    "Circle central bottom": (52.5, 34 - 9.15),
}

intersecciones_reales = {
    "Side line left": {
        "Side line bottom": (0.0, 0.0),
        "Side line top": (0.0, 68.0),
    },
    "Side line right": {
        "Side line bottom": (105.0, 0.0),
        "Side line top": (105.0, 68.0),
    },
    "Side line top": {
        "Side line left": (0.0, 68.0),
        "Side line right": (105.0, 68.0),
    },
    "Side line bottom": {
        "Side line left": (0.0, 0.0),
        "Side line right": (105.0, 0.0),
    },
    "Middle line": {
        "Side line bottom": (52.5, 0.0),
        "Side line top": (52.5, 68.0),
    },
    "Big rect. left main": {
        "Big rect. left bottom": (16.5, 13.85),
        "Big rect. left top": (16.5, 54.15),
    },
    "Big rect. left top": {
        "Big rect. left main": (16.5, 54.15),
        "Side line left": (0.0, 54.15),
    },
    "Big rect. left bottom": {
        "Big rect. left main": (16.5, 13.85),
        "Side line left": (0.0, 13.85),
    },
    "Big rect. right main": {
        "Big rect. right bottom": (88.5, 13.85),
        "Big rect. right top": (88.5, 54.15),
    },
    "Big rect. right top": {
        "Big rect. right main": (88.5, 54.15),
        "Side line right": (105.0, 54.15),
    },
    "Big rect. right bottom": {
        "Big rect. right main": (88.5, 13.85),
        "Side line right": (105.0, 13.85),
    },
    "Small rect. left main": {
        "Small rect. left bottom": (5.5, 24.84),
        "Small rect. left top": (5.5, 43.16),
    },
    "Small rect. left top": {
        "Small rect. left main": (5.5, 43.16),
        "Side line left": (0.0, 43.16),
    },
    "Small rect. left bottom": {
        "Small rect. left main": (5.5, 24.84),
        "Side line left": (0.0, 24.84),
    },
    "Small rect. right main": {
        "Small rect. right bottom": (99.5, 24.84),
        "Small rect. right top": (99.5, 43.16),
    },
    "Small rect. right top": {
        "Small rect. right main": (99.5, 43.16),
        "Side line right": (105.0, 43.16),
    },
    "Small rect. right bottom": {
        "Small rect. right main": (99.5, 24.84),
        "Side line right": (105.0, 24.84),
    },
}

def segment_to_line(x1, y1, x2, y2, name):
    """Convierte un segmento de línea en la forma Ax + By + C = 0."""
    A = y2 - y1
    B = x1 - x2
    C = x2 * y1 - x1 * y2
    return A, B, -C, name

def intersection(line1, line2):
    A1, B1, C1, NAME1 = line1
    A2, B2, C2, NAME2 = line2
    D = A1 * B2 - A2 * B1
    if D == 0:
        return None  # Son paralelas
    x = (B2 * C1 - B1 * C2) / D
    y = (A1 * C2 - A2 * C1) / D
    return x, y, NAME1, NAME2

#Lineas de campo
field_lines_map = []
for name, coords in field_map.items():
    if isinstance(coords, tuple):  # es una línea
        x1, y1, x2, y2 = coords
        field_lines_map.append(segment_to_line(x1, y1, x2, y2, name))

if not cap.isOpened():
    print("No se pudo abrir el video.")
    exit()
# Rango para el color blanco
lower_white = np.array([30, 10, 210])
upper_white = np.array([110, 70, 255])
prev_frame = None
total_frames=0
total_frames_homografiados=0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False)
    lines = model_line.predict(frame, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False)
    inclinaciones = model_inclinacion.predict(frame, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False)
    annotated_frame = frame.copy()
    positions = []
    index = 0
    for result in results:
        boxes = result.boxes
        detections = []
        for box in boxes:
            cls_id = int(box.cls[0])  
            if model.names[cls_id] == 'person':
                x1, y1, x2, y2 = box.xyxy[0].tolist()  
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                person_section = annotated_frame[y1:y2, x1:x2]
                small_section = cv2.resize(person_section, (20, 20), interpolation=cv2.INTER_LINEAR)
                hsv_section = cv2.cvtColor(small_section, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv_section, np.array([35, 40, 40]), np.array([85, 255, 255]))
                non_green_pixels = hsv_section[mask == 0]
                quantized_pixels = (non_green_pixels // 32) * 32
                pixels = quantized_pixels.reshape(-1, 3)
                pixels_list = [tuple(pixel) for pixel in pixels]
                if len(pixels_list) > 0:
                    most_common = Counter(pixels_list).most_common(1)[0][0]
                    domimant_color_rgb = cv2.cvtColor(np.uint8([[most_common]]), cv2.COLOR_HSV2BGR)[0][0]
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), tuple(int(c) for c in domimant_color_rgb), 2)
                    cv2.putText(annotated_frame, str(index), (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, tuple(int(c) for c in domimant_color_rgb), 2)
                    positions.append({"pos":((x1+x2)/2, max(y1, y2)),"type": model.names[cls_id], "index": index, "color": tuple(int(c) for c in domimant_color_rgb)})
                else:
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(annotated_frame, str(index), (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    positions.append({"pos":((x1+x2)/2, max(y1, y2)),"type": model.names[cls_id], "index": index, "color": (0, 255, 0)})
                index += 1
            #balon
            if model.names[cls_id] == 'sports ball':
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                positions.append({"pos":((x1+x2)/2, max(y1, y2)),"type": model.names[cls_id], "index": index, "color": (0, 0, 255)})
                index += 1
                
    inclinaciones_boxes = []
    for inclinacion in inclinaciones:
        boxes = inclinacion.boxes
        for box in boxes:
            cls_id = int(box.cls[0])
            if model_inclinacion.names[cls_id] == "unknown":
                continue
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
            inclinaciones_boxes.append({"pos":(x1, y1, x2, y2),"type": model_inclinacion.names[cls_id]})       
                
    # Dibuja las líneas detectadas
    field_lines = []
    central_circle = None
    for line in lines:
        boxes = line.boxes
        for box in boxes:
            cls_id = int(box.cls[0])
            line_name = model_line.names[cls_id]
            if line_name == "Circle central":
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
                central_circle = [x1, y1, x2, y2]
            elif line_name in ['Circle right','Circle left']:
                #dibujar rectangulo
                pass
            elif line_name in ["Goal right crossbar","Goal right post left","Goal right post right","Goal left post left","Goal left post left ","Goal left post left","Goal left post right","Goal left crossbar", 'unknown']:
                pass
            else:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                # Buscar la inclinación_box con mayor área de intersección
                max_iou = 0
                best_inclinacion = None
                for inc_box in inclinaciones_boxes:
                    ix1, iy1, ix2, iy2 = inc_box["pos"]
                    # calcular intersección
                    inter_x1 = max(x1, ix1)
                    inter_y1 = max(y1, iy1)
                    inter_x2 = min(x2, ix2)
                    inter_y2 = min(y2, iy2)
                    inter_w = max(0, inter_x2 - inter_x1)
                    inter_h = max(0, inter_y2 - inter_y1)
                    inter_area = inter_w * inter_h
                    area_line = (x2 - x1) * (y2 - y1)
                    area_inc = (ix2 - ix1) * (iy2 - iy1)
                    union_area = area_line + area_inc - inter_area
                    iou = inter_area / union_area if union_area > 0 else 0
                    if iou > max_iou:
                        max_iou = iou
                        best_inclinacion = inc_box
                # Si hay inclinación asociada, usar su sentido
                if best_inclinacion is not None:
                    sentido = best_inclinacion["type"]
                    if sentido == "r":
                        cv2.line(annotated_frame, (x1, y2), (x2, y1), (0, 255, 0), 2)
                        field_lines.append({"pos":(x1, y2, x2, y1),"type": line_name})
                    else:
                        cv2.line(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        field_lines.append({"pos":(x1, y1, x2, y2),"type": line_name})
                else:
                    #ponemos un rectangulo en la posicion de la linea
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                cv2.putText(annotated_frame, line_name, (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)                
                    
    intersections = []
    names = []           
    for i, line1 in enumerate(field_lines):
        for j, line2 in enumerate(field_lines):
            if i >= j:
                continue
            if line1 != line2:
                pos1 = line1["pos"]
                pos2 = line2["pos"]
                # Convertir las coordenadas de la línea a la forma Ax + By + C = 0
                A1, B1, C1, NAME1 = segment_to_line(pos1[0], pos1[1], pos1[2], pos1[3], line1["type"])
                A2, B2, C2, NAME2 = segment_to_line(pos2[0], pos2[1], pos2[2], pos2[3], line2["type"])
                # Calcular la intersección
                intersection_point = intersection((A1, B1, C1, NAME1), (A2, B2, C2, NAME2))
                if intersection_point is not None:
                    x, y, name1, name2 = intersection_point
                    margin = 150
                    if x > 0 and y >0 and x < frame.shape[1] and y < frame.shape[0]:
                        intersections.append({"pos":(x, y),"lines": (name1, name2)})
                        names.append((name1, name2))
                        cv2.circle(annotated_frame, (int(intersection_point[0]), int(intersection_point[1])), 5, (255, 255, 0), -1)
                    elif x> 0 - margin and y >0 - margin and x < frame.shape[1] + margin and y < frame.shape[0] + margin:
                        intersections.append({"pos":(x, y),"lines": (name1, name2)})
                        names.append((name1, name2))
                        
    #si faltan intersecciones evaluar si hay lineas que estan completas y calcular la intersección que debería haber
    if len(intersections) < 4:
        margin = 5
        for line in field_lines:
            x1, y1, x2, y2 = line["pos"]
            #evalua que la linea este completa (ningun extremo cercano a los bordes de la imagen)
            if (x1 > margin and x2 < frame.shape[1] - margin) and (y1 > margin and y2 < frame.shape[0] - margin):
                # ver tipo de linea
                type = line["type"]
                #obtener intersecciones reales
                if type in intersecciones_reales:
                    interseccion_type = intersecciones_reales[type]
                    keys = list(interseccion_type.keys())
                    # si la linea es horizontal, asignamos el punto más alto como el punto de intersección más alto
                    if interseccion_type[keys[0]][1] == interseccion_type[keys[1]][1]:
                        # la intersección es horizontal
                        #evaluamos que punto de la imagen esta mas a la izquierda
                        if x1 < x2:
                            if interseccion_type[keys[0]][0] < interseccion_type[keys[1]][0]:
                                # la intersección es la primera
                                intersections.append({"pos":(x1, y1),"lines": (type, keys[0])})
                                intersections.append({"pos":(x2, y2),"lines": (type, keys[1])})
                            else:
                                # la intersección es la segunda
                                intersections.append({"pos":(x1, y1),"lines": (type, keys[1])})
                                intersections.append({"pos":(x2, y2),"lines": (type, keys[0])})
                        else:
                            if interseccion_type[keys[0]][0] < interseccion_type[keys[1]][0]:
                                # la intersección es la primera
                                intersections.append({"pos":(x1, y1),"lines": (type, keys[1])})
                                intersections.append({"pos":(x2, y2),"lines": (type, keys[0])})
                            else:
                                # la intersección es la segunda
                                intersections.append({"pos":(x1, y1),"lines": (type, keys[0])})
                                intersections.append({"pos":(x2, y2),"lines": (type, keys[1])})
                    else:
                        # la intersección es vertical
                        #evaluamos que punto de la imagen es más alto
                        if y1 < y2:
                            if interseccion_type[keys[0]][1] > interseccion_type[keys[1]][1]:
                                # la intersección es la primera
                                intersections.append({"pos":(x1, y1),"lines": (type, keys[0])})
                                intersections.append({"pos":(x2, y2),"lines": (type, keys[1])})
                            else:
                                # la intersección es la segunda
                                intersections.append({"pos":(x1, y1),"lines": (type, keys[1])})
                                intersections.append({"pos":(x2, y2),"lines": (type, keys[0])})
                        else:
                            if interseccion_type[keys[0]][1] > interseccion_type[keys[1]][1]:
                                # la intersección es la primera
                                intersections.append({"pos":(x1, y1),"lines": (type, keys[1])})
                                intersections.append({"pos":(x2, y2),"lines": (type, keys[0])})
                            else:
                                # la intersección es la segunda
                                intersections.append({"pos":(x1, y1),"lines": (type, keys[0])})
                                intersections.append({"pos":(x2, y2),"lines": (type, keys[1])})
                    names.append((type, keys[0]))
                    names.append((type, keys[1])) 

    # Intersección de líneas del campo        
    intersections_field = []
    for i, line1 in enumerate(field_lines_map):
        for j, line2 in enumerate(field_lines_map):
            if i >= j:
                continue
            point = intersection(line1, line2)
            if point is not None:
                x, y, name1, name2 = point
                if (name1, name2) in names or (name2, name1) in names:
                    intersections_field.append({
                        "pos": (x, y),
                        "lines": (name1, name2)
                    })        
           
    matched_pairs = []
    for i in intersections:
        for f in intersections_field:
            if set(i["lines"]) == set(f["lines"]):
                matched_pairs.append((i["pos"], f["pos"]))  # (imagen, cancha)
                break 
            
    #si me faltan matched pair y tengo el circulo central
    #if len(matched_pairs) < 4 and central_circle is not None:
    if central_circle is not None:
        x1, y1, x2, y2 = central_circle
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        max_x = max(x1, x2)
        min_x = min(x1, x2)
        midle_line  = None
        #obtenemos la linea media
        for line in field_lines:
            if line["type"] == "Middle line":
                midle_line = line
                break
        #si la linea media no es None y el circulo central no choca ni con el borde inferior ni superior
        if midle_line is not None and y1 > 5 and y2 < frame.shape[0] - 5:
            x1_ml, y1_ml, x2_ml, y2_ml = midle_line["pos"]
            x1_ml, y1_ml, x2_ml, y2_ml = map(int, (x1_ml, y1_ml, x2_ml, y2_ml))
            cv2.circle(annotated_frame, (int(x1_ml), int(y1_ml)), 5, (0, 255, 0), -1)
            #obtenemos la interseccion entre la linea media y el top del circulo
            line1 = segment_to_line(x1_ml, y1_ml, x2_ml, y2_ml, "Middle line")
            line2 = segment_to_line(x1, y2, x2, y2, "Circle central bottom")
            line3 = segment_to_line(x1, y1, x2, y1, "Circle central top")
            #buscamos las intersecciones
            intersection1 = intersection(line1, line2) # interseccion entre la linea media y el top del circulo
            intersection2 = intersection(line1, line3) # interseccion entre la linea media y el bottom del circulo
            if intersection1 is not None:
                x, y, name1, name2 = intersection1
                cv2.circle(annotated_frame, (int(x), int(y)), 5, (0, 0, 255), -1)
                matched_pairs.append(((x, y), (52.5, 34-9.15)))
            if intersection2 is not None:
                x, y, name1, name2 = intersection2
                cv2.circle(annotated_frame, (int(x), int(y)), 5, (0, 0, 255), -1)
                matched_pairs.append(((x, y), (52.5, 34+9.15)))
            #obtenemos el punto medio entre las intersecciones
            if intersection1 is not None and intersection2 is not None:
                xi1, yi1, _, _ = intersection1
                xi2, yi2, _, _ = intersection2
                #dibujamos la linea entre las intersecciones
                cv2.line(annotated_frame, (int(xi1), int(yi1)), (int(xi2), int(yi2)), (0, 0, 255), 2)
                #obtenemos el punto medio
                center_x = int((xi1 + xi2) / 2)
                center_y = int((yi1 + yi2) / 2)
                #dibujamos el circulo
                cv2.circle(annotated_frame, (int(center_x), int(center_y)), 5, (0, 0, 255), -1)
                matched_pairs.append(((center_x, center_y), (52.5, 34)))
                #si el lado mas a la derecha no esta en el borde hacemos una linea
                if max_x < frame.shape[1] - 5:
                    cv2.circle(annotated_frame, (max_x, center_y), 5, (0, 0, 255), -1)
                    matched_pairs.append(((max_x, center_y), (52.5 + 9.15, 34)))
                if min_x > 5:
                    #dibujamos la linea
                    cv2.circle(annotated_frame, (min_x, center_y), 5, (0, 0, 255), -1)
                    matched_pairs.append(((min_x, center_y), (52.5 - 9.15, 34)))
            
    # Homografía 2d
    positions_homography = []
    intersection_homography = []        
    if len(matched_pairs) > 4:
        
        src_pts = np.array([pt[0] for pt in matched_pairs], dtype=np.float32)
        dst_pts = np.array([pt[1] for pt in matched_pairs], dtype=np.float32)
        
        # Calcular la homografía
        H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)
        if H is not None:
            # Aplicar la homografía a los puntos de la imagen
            for pos in positions:
                try:
                    point = np.array([pos["pos"][0], pos["pos"][1], 1], dtype=np.float32)
                    transformed_point = np.dot(H, point)
                    transformed_point /= transformed_point[2]  # Normalizarq
                    positions_homography.append({"pos":(transformed_point[0], transformed_point[1]),"type": pos["type"], "index": pos["index"], "color": pos["color"]})
                except Exception as e:
                    print(f"Error al aplicar homografía a la posición {pos['index']}: {e}")
                    continue
            # Aplicar la homografía a los puntos de intersección
            for pos in intersections:
                try:
                    point = np.array([pos["pos"][0], pos["pos"][1], 1], dtype=np.float32)
                    transformed_point = np.dot(H, point)
                    transformed_point /= transformed_point[2]  # Normalizar
                    intersection_homography.append({"pos":(transformed_point[0], transformed_point[1]),"lines": pos["lines"]})     
                except Exception as e:
                    print(f"Error al aplicar homografía a la posición {pos['lines']}: {e}")
                    continue           
            # Draw minimap of the field
            # Crear una imagen en blanco para el minimapa del campo
            field_map_img = np.ones((800, 1200, 3), dtype=np.uint8) * 255

            # Escalado para ajustar el campo a la imagen
            scale_x = 10  # 1 metro = 10 pixeles
            scale_y = 10
            offset_x = 50  # margen izquierdo
            offset_y = 50  # margen superior

            def to_img_coords(x, y):
                # Convierte coordenadas de metros a pixeles en la imagen
                return int(x * scale_x + offset_x), int(field_map_img.shape[0] - (y * scale_y + offset_y))

            # Dibujar líneas y círculos del campo
            for name, coords in field_map.items():
                if isinstance(coords, tuple):  # Línea
                    x1, y1, x2, y2 = coords
                    pt1 = to_img_coords(x1, y1)
                    pt2 = to_img_coords(x2, y2)
                    cv2.line(field_map_img, pt1, pt2, (255, 0, 0), 2)
                    if name == "Middle line":
                        center_x = x1
                        center_y = (y1 + y2) / 2
                        center_pt = to_img_coords(center_x, center_y)
                        cv2.circle(field_map_img, center_pt, 5, (255, 0, 0), -1)
                elif isinstance(coords, dict):  # Círculo
                    center = to_img_coords(coords["center"][0], coords["center"][1])
                    radius = int(coords["radius"] * scale_x)
                    cv2.circle(field_map_img, center, radius, (0, 255, 0), 2)

            # Dibujar posiciones proyectadas por homografía
            for pos in positions_homography:
                px, py = to_img_coords(pos["pos"][0], pos["pos"][1])
                color = pos["color"]
                cv2.circle(field_map_img, (px, py), 7, color, -1)
                cv2.putText(field_map_img, str(pos["index"]), (px-10, py-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            cv2.putText(field_map_img, "Minimapa del campo y posiciones proyectadas", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
            # Mostrar el campo
            total_frames_homografiados += 1
            cv2.imshow("Campo", field_map_img)                  
                    
                    
    # Mostrar la imagen con las detecciones
    total_frames += 1
    cv2.imshow("Detecciones", annotated_frame)
    print(f"Frames homografiados: {total_frames_homografiados}/{total_frames} -> {total_frames_homografiados/total_frames*100:.2f}%")
    #esperamos una tecla, si es 'q' salimos si no esperamos que sea espacio
    """ key = cv2.waitKey(0)
    if key == 32:  # Espacio
        continue
    elif key == ord('q'):
        break """
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #si el video se termina, salimos
    if total_frames >= cap.get(cv2.CAP_PROP_FRAME_COUNT):
        break
cap.release()
cv2.destroyAllWindows()

Frames homografiados: 1/1 -> 100.00%
Frames homografiados: 1/2 -> 50.00%
Frames homografiados: 1/3 -> 33.33%
Frames homografiados: 1/4 -> 25.00%
Frames homografiados: 1/5 -> 20.00%
Frames homografiados: 1/6 -> 16.67%
Frames homografiados: 1/7 -> 14.29%
Frames homografiados: 1/8 -> 12.50%
Frames homografiados: 1/9 -> 11.11%
Frames homografiados: 1/10 -> 10.00%
Frames homografiados: 2/11 -> 18.18%
Frames homografiados: 3/12 -> 25.00%
Frames homografiados: 3/13 -> 23.08%
Frames homografiados: 3/14 -> 21.43%
Frames homografiados: 3/15 -> 20.00%
Frames homografiados: 3/16 -> 18.75%
Frames homografiados: 3/17 -> 17.65%
Frames homografiados: 3/18 -> 16.67%
Frames homografiados: 3/19 -> 15.79%
Frames homografiados: 3/20 -> 15.00%
Frames homografiados: 3/21 -> 14.29%
Frames homografiados: 3/22 -> 13.64%
Frames homografiados: 3/23 -> 13.04%
Frames homografiados: 3/24 -> 12.50%
Frames homografiados: 3/25 -> 12.00%
Frames homografiados: 3/26 -> 11.54%
Frames homografiados: 3/27 -> 11.11%
Frames ho